# Transforming

In this notebook, we will explore how to use Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

## Setup

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')
MODEL=os.getenv('LLM_MODEL')

In [2]:
# For OpenAI after 1.0.0 (1.13.3 at the time of writing)
client = openai.OpenAI()

def get_completion(prompt, model=MODEL):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

## Translation

ChatGPT is trained with sources in many languages. This gives the model the ability to do translation. Here are some examples of how to use this capability.

In [3]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)

Hola, me gustaría pedir una licuadora.


In [4]:
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

The language is French. The sentence "Combien coûte le lampadaire?" translates to "How much does the floor lamp cost?" in English.


In [5]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

Formal:
¿Le gustaría pedir una almohada?

Informal:
¿Te gustaría pedir una almohada?


## Try it yourself!
Try some translations on your own!

### Universal Translator
Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!

In [6]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [7]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message (The language of the sentence "La performance du système est plus lente que d'habitude." is French. It translates to "The system's performance is slower than usual." in English.): La performance du système est plus lente que d'habitude.
English: The system performance is slower than usual.

Korean: 시스템 성능이 평소보다 느립니다. 

Original message (The language of the sentence "Mi monitor tiene píxeles que no se iluminan." is Spanish. The sentence translates to "My monitor has pixels that do not light up." in English.): Mi monitor tiene píxeles que no se iluminan.
English:
"My monitor has pixels that do not light up."

Korean:
"내 모니터에는 불이 들어오지 않는 픽셀이 있습니다." 

Original message (The language is Italian. The sentence "Il mio mouse non funziona" translates to "My mouse does not work" in English.): Il mio mouse non funziona
English: My mouse is not working.
Korean: 마우스가 작동하지 않아요. 

Original message (The language of the sentence "Mój klawisz Ctrl jest zepsuty" is Polish. Translated to E

## Tone Transformation
Writing can vary based on the intended audience. ChatGPT can produce different tones.

In [8]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)

Dear [Recipient's Name],

I hope this message finds you well. My name is Joe, and I am writing to bring your attention to the specifications of a particular standing lamp that may be of interest to you. I believe it could be a valuable addition to your selection.

Please find attached the detailed specifications for your review. I am confident that you will find the features of this lamp to be both innovative and beneficial for your needs.

Should you have any questions or require further information, please do not hesitate to contact me. I am looking forward to your feedback and am available at your earliest convenience for a discussion.

Thank you for considering this product. I am eager to hear your thoughts on it.

Warm regards,

Joe [Your Last Name]
[Your Position]
[Your Contact Information]


## Format Conversion
ChatGPT can translate between formats. The prompt should describe the input and output formats.

In [9]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

To translate the provided Python dictionary into an HTML table with column headers and a title, you can use the following HTML code:

```html
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Restaurant Employees</title>
<style>
    table {
        width: 100%;
        border-collapse: collapse;
    }
    table, th, td {
        border: 1px solid black;
    }
    th, td {
        padding: 8px;
        text-align: left;
    }
    th {
        background-color: #f2f2f2;
    }
</style>
</head>
<body>

<h2>Restaurant Employees</h2>

<table>
    <tr>
        <th>Name</th>
        <th>Email</th>
    </tr>
    <tr>
        <td>Shyam</td>
        <td>shyamjaiswal@gmail.com</td>
    </tr>
    <tr>
        <td>Bob</td>
        <td>bob32@gmail.com</td>
    </tr>
    <tr>
        <td>Jai</td>
        <td>jai87@gmail.com</td>
    </tr>
</table>

</body>
</html>
```

This HTML code creates a simple table with 

In [10]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


## Spellcheck/Grammar check.

Here are some examples of common grammar and spelling problems and the LLM's response. 

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [11]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

The girl with the black and white puppies has a ball.
No errors found.
It's going to be a long day. Does the car need its oil changed?
There goes my freedom. They're going to bring their suitcases.
You're going to need your notebook.
That medicine affects my ability to sleep. Have you heard of the butterfly effect?
This phrase is to check ChatGPT for spelling ability.


In [12]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

I purchased this for my daughter as a birthday gift because she constantly borrows mine from my room. Indeed, adults can be fond of pandas as well. She carries it with her everywhere, and it's incredibly soft and adorable. However, one of the ears is slightly lower than the other, which I suspect was not an intentional design choice for asymmetry. The size is a bit disappointing considering the price; I believe there may be larger alternatives available for the same cost. On a positive note, it arrived a day earlier than anticipated, allowing me some time to enjoy it myself before presenting it to my daughter.


In [13]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style="color:red;font-weight:700;text-decoration:line-through;">Got </span><span style="color:red;font-weight:700;">I purchased </span>this for my daughter <span style="color:red;font-weight:700;text-decoration:line-through;">for her </span><span style="color:red;font-weight:700;">as a </span>birthday <span style="color:red;font-weight:700;text-decoration:line-through;">cuz </span><span style="color:red;font-weight:700;">gift because </span>she <span style="color:red;font-weight:700;text-decoration:line-through;">keeps taking </span><span style="color:red;font-weight:700;">constantly borrows </span>mine from my <span style="color:red;font-weight:700;text-decoration:line-through;">room.  Yes, </span><span style="color:red;font-weight:700;">room. Indeed, </span>adults <span style="color:red;font-weight:700;text-decoration:line-through;">also like </span><span style="color:red;font-weight:700;">can be fond of </span>pandas <span style="color:red;font-weight:700;text-decoration:line-through;">too.  </span><span style="color:red;font-weight:700;">as well. </span>She <span style="color:red;font-weight:700;text-decoration:line-through;">takes </span><span style="color:red;font-weight:700;">carries </span>it <span style="color:red;font-weight:700;text-decoration:line-through;">everywhere </span>with <span style="color:red;font-weight:700;text-decoration:line-through;">her, </span><span style="color:red;font-weight:700;">her everywhere, </span>and it's <span style="color:red;font-weight:700;text-decoration:line-through;">super </span><span style="color:red;font-weight:700;">incredibly </span>soft and <span style="color:red;font-weight:700;text-decoration:line-through;">cute.  One </span><span style="color:red;font-weight:700;">adorable. However, one </span>of the ears is <span style="color:red;font-weight:700;text-decoration:line-through;">a bit </span><span style="color:red;font-weight:700;">slightly </span>lower than the other, <span style="color:red;font-weight:700;text-decoration:line-through;">and </span><span style="color:red;font-weight:700;">which </span>I <span style="color:red;font-weight:700;text-decoration:line-through;">don't think that </span><span style="color:red;font-weight:700;">suspect </span>was <span style="color:red;font-weight:700;text-decoration:line-through;">designed to be asymmetrical. It's </span><span style="color:red;font-weight:700;">not an intentional design choice for asymmetry. The size is </span>a bit <span style="color:red;font-weight:700;text-decoration:line-through;">small for what </span><span style="color:red;font-weight:700;">disappointing considering the price; </span>I <span style="color:red;font-weight:700;text-decoration:line-through;">paid for it though. I think </span><span style="color:red;font-weight:700;">believe </span>there <span style="color:red;font-weight:700;text-decoration:line-through;">might </span><span style="color:red;font-weight:700;">may </span>be <span style="color:red;font-weight:700;text-decoration:line-through;">other options that are bigger </span><span style="color:red;font-weight:700;">larger alternatives available </span>for the same <span style="color:red;font-weight:700;text-decoration:line-through;">price.  It </span><span style="color:red;font-weight:700;">cost. On a positive note, it </span>arrived a day earlier than <span style="color:red;font-weight:700;text-decoration:line-through;">expected, so I got </span><span style="color:red;font-weight:700;">anticipated, allowing me some time </span>to <span style="color:red;font-weight:700;text-decoration:line-through;">play with </span><span style="color:red;font-weight:700;">enjoy </span>it myself before <span style="color:red;font-weight:700;text-decoration:line-through;">I gave </span><span style="color:red;font-weight:700;">presenting </span>it to my daughter.

## Try it yourself!
Try changing the instructions to form your own review.